In [1]:
import numpy as np

from sklearn import datasets
import sklearn
import sklearn.decomposition
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from sklearn import linear_model 

from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd
from pymystem3 import Mystem

In [2]:
import pymorphy2
import functools
import operator
import re
from tqdm import tqdm_notebook
import numpy as np
import gensim
from gensim.models import Word2Vec

from collections import Counter
import logging

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\Nina\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
df1 = pd.read_csv('negative.csv', sep=';', header=None, encoding='utf-8',\
                  names=['id1','id2','nick','comment','index1','index2','index3','index4','index5','index6','index7','index8'])
df2 = pd.read_csv('positive.csv', sep=';', encoding = 'utf-8', header=None, \
                 names=['id1','id2','nick','comment','index1','index2','index3','index4','index5','index6','index7','index8'])
df1['index'] = 0
df2['index'] = 1
df = df1.append(df2)


al = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

def no_pun(s):
    return ''.join(x for x in s.lower() if x in al)

m = Mystem()

def lem(text):
    lemmas = m.lemmatize(text)
    return ''.join(lemmas)
lem('Привет!! как дела?'), no_pun('Привет!! как дела?')

('привет!! как дело?\n', 'привет как дела')

In [5]:
#pos = df2[3]
#print(pos.shape)
#neg = df1[3]
#print(neg.shape)

(114911,)
(111923,)


In [6]:
%%time
morph = pymorphy2.MorphAnalyzer()

df['clear_comment'] = df.comment.apply(\
                            lambda text: morph.parse(re.sub('[^а-яА-Я ]', '', ''.join(str(text)).lower()))[0].normal_form)

Wall time: 3min 31s


In [7]:
class Splitting():
    def __init__(self, sentences):
        self.sentences = sentences
        if '' in self.sentences:
            self.sentences.remove('')

    def __iter__(self):
        return (s.split() for s in self.sentences)

In [8]:
def splitting(sentences):
    if '' in sentences:
        sentences.remove('')
    return (s.split() for s in sentences)

In [33]:
%%time
words = Splitting(list(df.clear_comment))
model = Word2Vec(words, min_count=30, hs = 0, workers = 4, sg=0, iter = 50)

Wall time: 5min 7s


In [46]:
# пример работы ворд2века: модель обучена нормально
words = model.wv.index2word

In [56]:
%%time
vectors = pd.DataFrame(model.wv.vectors)
vectors['words'] = words
vectorizer = TfidfVectorizer(min_df = 30)
texts_tfidf = vectorizer.fit_transform(df.clear_comment)

Wall time: 13.1 s


In [35]:
%%time
fn = vectorizer.get_feature_names()

Wall time: 14 ms


In [ ]:
mt = texts_tfidf.todense()

In [47]:
word = pd.DataFrame(fn, columns = ['words'])

In [57]:
word.shape

(6203, 1)

In [58]:
f = pd.merge(word, vectors, on = 'words')

In [55]:
vectors.shape

(6278, 101)

In [59]:
f.shape

(6203, 101)

In [60]:
vecs = f.iloc[:,1:].values.transpose()

In [61]:
vecs = np.matrix(vecs)

In [62]:
vecs

matrix([[-0.3912025 ,  0.4128594 ,  0.80275023, ..., -0.21250527,
         -0.63431686,  0.5096467 ],
        [ 0.98595387,  0.39626396, -0.31937283, ..., -0.32830134,
         -0.8976158 , -0.09088632],
        [ 1.1960278 ,  0.37108126, -0.95495933, ..., -0.41504523,
         -0.25617343,  1.4324073 ],
        ...,
        [ 1.0194621 ,  1.0912251 ,  0.3914889 , ...,  0.4198572 ,
         -1.281239  ,  1.3694355 ],
        [ 2.143502  ,  0.762172  ,  1.7059977 , ..., -0.05147666,
          0.08823078, -0.9549198 ],
        [-1.5098135 , -0.19606103, -0.153078  , ...,  1.2810558 ,
         -0.5516037 , -0.90586656]], dtype=float32)

In [63]:
mt.shape

(226834, 6203)

In [64]:
fm = vecs*(mt.transpose())

In [65]:
X_train = fm.transpose()

In [67]:
%%time
clusters = KMeans(n_clusters = 10).fit_predict(X_train)

Wall time: 6min 54s


In [68]:
df['cluster'] = clusters

In [69]:
def pos(word, morth = pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}

In [74]:
for clust in df['cluster'].unique():
    lst = [item for sublist in df[df['cluster'] ==clust]['clear_comment'].apply(lambda x: x.split()) for item in sublist]
    cnt = Counter(lst)
    print('cluster = '+ str(clust))
    print('rows amount: '+ str(df[df['cluster'] == clust].shape[0]))
    print('key words + frequency')
    print(*[word for word in cnt.most_common(100) if pos(word[0]) not in functors_pos][:30])
    print()

cluster = 6
rows amount: 20222
key words + frequency
('я', 3981) ('день', 2600) ('уже', 2156) ('сегодня', 1808) ('меня', 1649) ('год', 1558) ('все', 1420) ('новый', 1339) ('будет', 1306) ('завтра', 1289) ('года', 996) ('нас', 856) ('мы', 848) ('раз', 725) ('скоро', 714) ('дня', 709) ('нг', 612) ('мне', 604) ('нет', 584) ('буду', 512) ('два', 451) ('первый', 447) ('теперь', 443) ('всего', 428) ('последний', 427) ('хочу', 425) ('недели', 417) ('очень', 408) ('настроение', 403) ('ещ', 396)

cluster = 9
rows amount: 20398
key words + frequency
('я', 3566) ('все', 2653) ('они', 2179) ('меня', 2054) ('мы', 1785) ('вы', 1628) ('такие', 1335) ('люди', 1158) ('мне', 1142) ('нас', 1135) ('их', 994) ('мои', 883) ('вас', 744) ('уже', 708) ('кто', 704) ('были', 677) ('там', 662) ('которые', 655) ('нет', 635) ('всех', 602) ('почему', 588) ('эти', 577) ('людей', 574) ('них', 519) ('очень', 500) ('будут', 462) ('тоже', 441) ('какие', 435) ('ты', 427) ('люблю', 418)

cluster = 3
rows amount: 11514
key 